In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import numpy as np

Using TensorFlow backend.


In [2]:
IMG_SIZE = 300

In [3]:
import json
def readJSON(file):
    with open(file) as f:
        return json.load(f)

In [4]:
train = readJSON('food41/meta/train.json')
test = readJSON('food41/meta/test.json')

In [ ]:
from PIL import Image as img
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
def loadImage(file):
    return np.array([np.array(img.open('food41/images/'+i+'.jpg').resize((IMG_SIZE, IMG_SIZE), img.ANTIALIAS)) for i in file])

In [ ]:
batch_size = 5
num_classes = 6
epochs = 2

# tidak boleh
ice_cream_train = loadImage(train.get('ice_cream'))
ice_cream_test = loadImage(test.get('ice_cream'))
print(0)
donuts_train = loadImage(train.get('donuts'))
donuts_test = loadImage(test.get('donuts'))
print(1)
french_fries_train = loadImage(train.get('french_fries'))
french_fries_test = loadImage(test.get('french_fries'))
print(2)

# boleh
waffles_train = loadImage(train.get('waffles'))
waffles_test = loadImage(test.get('waffles'))
print(6)
miso_soup_train = loadImage(train.get('miso_soup'))
miso_soup_test = loadImage(test.get('miso_soup'))
print(7)
sushi_train = loadImage(train.get('sushi'))
sushi_test = loadImage(test.get('sushi'))
print(8)



0
1
2
6
7
8


In [ ]:
x_train = np.concatenate((
    ice_cream_train,
    donuts_train,
    french_fries_train,
    waffles_train,
    miso_soup_train,
    sushi_train),
    axis=0
)
print("done")
x_test = np.concatenate((
    ice_cream_test,
    donuts_test,
    french_fries_test,
    waffles_test,
    miso_soup_test,
    sushi_test),
    axis=0
)
print("done")
# generate y_train
i = 0
y_train = []
count = 1
while (i < num_classes):
    if (count <= 750):
        y_train.append(i)
        count += 1
    else:
        i += 1
        count=1
print("done")
i = 0
y_test = []
count = 1
while (i < num_classes):
    if (count <= 250):
        y_test.append(i)
        count += 1
    else:
        i += 1
        count=1
print("done")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

done
done
done
done


## CNN Model with ReLu Activation

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 296, 296, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 148, 148, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1401856)           0         
_________________________________________________________________
dense_1 (Dense)      

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 4500 samples, validate on 1500 samples
Epoch 1/2
4500/4500 [==============================] - 797s 177ms/step - loss: 9.2899 - acc: 0.2516 - val_loss: 8.5739 - val_acc: 0.2793
Epoch 2/2
4500/4500 [==============================] - 803s 178ms/step - loss: 8.7097 - acc: 0.2882 - val_loss: 8.7291 - val_acc: 0.2913
Test loss: 8.72913640276591
Test accuracy: 0.29133333333333333


## CNN Model with Sigmoid Activation

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='sigmoid',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Conv2D(64, (3, 3), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 296, 296, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 148, 148, 64)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1401856)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 8411142   
Total params: 8,430,534
Trainable params: 8,430,534
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 4500 samples, validate on 1500 samples
Epoch 1/2
4500/4500 [==============================] - 863s 192ms/step - loss: 8.6057 - acc: 0.1662 - val_loss: 8.6084 - val_acc: 0.1667
Epoch 2/2
4500/4500 [==============================] - 881s 196ms/step - loss: 8.6084 - acc: 0.1667 - val_loss: 8.6084 - val_acc: 0.1667
Test loss: 8.608353861490885
Test accuracy: 0.16666666666666666


## CNN Model with ELU Activation

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='elu',
                 input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(num_classes, activation='elu'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 296, 296, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 148, 148, 64)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1401856)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 8411142   
Total params: 8,430,534
Trainable params: 8,430,534
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 4500 samples, validate on 1500 samples
Epoch 1/2
4500/4500 [==============================] - 868s 193ms/step - loss: 13.4299 - acc: 0.1662 - val_loss: 13.4317 - val_acc: 0.1667
Epoch 2/2
4500/4500 [==============================] - 883s 196ms/step - loss: 13.4290 - acc: 0.1664 - val_loss: 13.4317 - val_acc: 0.1667
Test loss: 13.431746172587077
Test accuracy: 0.16666666666666666
